In [1]:
# Initialize
import simpy
import random

SIM_TIME = 50
PASSENGER_INTERVAL = 10

In [ ]:
#Passenger Generator
def passenger_generator(env):
    pid = 1
    while True:
        yield env.timeout(random.expovariate(1 / PASSENGER_INTERVAL))
        pickup = random.randint(0, 2)
        destination = random.randint(0,2)
        while pickup == destination:
            destination = random.randint(0,2)
        passenger = Passenger(pid, pickup, destination)
        print(f"[{env.now:.1f}] Passenger {pid} arrived, travel plan: floor {pickup}--> floor {destination}")


In [3]:
class Passenger:
    def __init__(self, pid, pickup, destination):
        self.id = pid
        self.pickup = pickup
        self.destination = destination
    
    def __repr__(self):
        return f"P{self.id}({self.pickup} --> {self.destination})"

In [7]:
env = simpy.Environment()
env.process(passenger_generator(env))
env.run(until = SIM_TIME)

[20.1] Passenger 1 just arrived, travel plan: floor 0--> floor 1
[27.8] Passenger 1 just arrived, travel plan: floor 0--> floor 1
[33.7] Passenger 1 just arrived, travel plan: floor 2--> floor 0
[40.7] Passenger 1 just arrived, travel plan: floor 0--> floor 2
[42.0] Passenger 1 just arrived, travel plan: floor 1--> floor 2
